# Caching data in nlmod
Groundwater flow models are often data-intensive. Execution times can be shortened
significantly by caching data. This notebooks explains how this caching is implemented
in `nlmod`. The first three sections explain how to use the caching in nlmod. The last
section contains more technical details on the implementation and limitations of
caching in nlmod.

In [ ]:
import os
import xarray as xr

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

## Cache directory

When you create a model you usually start by assigning a model workspace. This is a directory where model data is stored. The `nlmod.util.get_model_dirs()` function can be used to create a file structure in two steps:
1. The model workspace directory is created if it does not exist yet. 
2. Two subdirectories are created: 'figure' and 'cache'. 

Calling the function below we create the `figdir` and `cachedir` variables with the paths of the subdirectories. In this notebook we will use this `cachedir` to write and read cached data. It is possible to define your own cache directory.

In [ ]:
model_ws = "model5"

# Model directories
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)

print(model_ws)
print(figdir)
print(cachedir)

## Caching

In `nlmod` you can use the `get_combined_layer_models` function to obtain a layer model based on `regis`.

In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent=[95000.0, 105000.0, 494000.0, 500000.0], use_geotop=False
)

As you may notice, this function takes some time to complete because the data is downloaded and projected on the desired model grid. Everytime you run this function you have to wait for the process to finish which results in an unhealthy number of coffee breaks. This is why we use caching. To store our cache we use netCDF files. The `layer_model` variable is an `xarray.Dataset`. You can read/write an `xarray.Dataset` to/from a NetCDF file using the code below.

In [ ]:
# write netcdf with layer model data
layer_model.to_netcdf(os.path.join(cachedir, "layer_test.nc"))

In [ ]:
# read netcdf with layer model data
layer_model_from_cache = xr.open_dataset(
    os.path.join(cachedir, "layer_test.nc"), mask_and_scale=False, decode_coords="all"
)

In [ ]:
# compare cache with original
assert layer_model_from_cache.equals(layer_model)

Reading and writing netcdf files is the main principle behind caching in `nlmod`. We write the `layer_model` to a NetCDF file when we call the `get_combined_layer_models` function for the first time. The next time we call the function we can read the cached NetCDF file instead. This reduces exuction time signficantly. You can simply use this caching abilities by specifying a `cachedir` and a `cachename` in the function call.

In [ ]:
layer_model2 = nlmod.read.regis.get_combined_layer_models(
    extent=[95000.0, 105000.0, 494000.0, 500000.0],
    use_geotop=False,
    cachedir=cachedir,
    cachename="combined_layer_ds.nc",
)

## Caching steps

The netCDF caching is applied to a number of functions in nlmod that have an xarray dataset as output. When you call these functions using the `cachedir` and `cachename` arguments the following steps are taken:

1. See if there is a netCDF file with the specified cachename in the specified cache directory. If the file exists go to step 2, otherwise go to step 3.
2. Read the netCDF file and return as an xarray dataset if:
    1. The cached dataset was created using the same function arguments as the current function call. 
    2. The module where the function is defined has not been changed after the cache was created.
3. Run the function to obtain an xarray dataset. Save this dataset as a netCDF file, using the specified cachename and cache directory, for next time. Also return the dataset.

This is the flowchart of an ordinary function call:<br>
![image function call](img/ordinary_function_call.png)

This is the flowchart of a function call using the caching from nlmod:<br>
![image cache function call](img/cache_function_call.png)

### Caching functions

The following functions use the caching as described above:

- `nlmod.read.regis.get_combined_layer_models`
- `nlmod.read.regis.download_regis`
- `nlmod.read.rws.download_surface_water`
- `nlmod.read.rws.download_northsea`
- `nlmod.read.knmi.get_recharge`
- `nlmod.read.jarkus.download_bathymetry`
- `nlmod.read.geotop.download_geotop`
- `nlmod.read.ahn.download_ahn`

## Checking the cache
One of the steps in the caching process is to check if the cache was created using the same function arguments as the current function call. This check has some limitations:

- Only function arguments with certain types are checked. These types include: int, float, bool, str, bytes, list, tuple, dict, numpy.ndarray, xarray.DataArray, pandas.DataFrame, pandas.Series and xarray.Dataset. If a function argument has a different type the cache is never used. In future development more types may be added to the checks.
- If one of the function arguments is an xarray Dataset the check is somewhat different. Not all the data in a dataset it checked as it would take to much time. Therefore we use the `nlmod.cache.ds_contains` function to specify which coordinates, data variables and attributes should be checked. The arguments for this are specified in the cache decorator. Users of nlmod do not have to worry about these.
- It is not possible to cache the results of a function with more than one xarray Dataset as an argument. This is due to the difference in checking datasets. If more than one xarray dataset is given the cache decoraters raises a TypeError.
- If one of the function arguments is a filepath of type str we only check if the cached filepath is the same as the current filepath. We do not check if any changes were made to the file after the cache was created.

You can test how the caching works in different situations by running the function below a few times with different function arguments. The logs provide some information about using the cache or not.

In [ ]:
# layer model
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent=[95000.0, 105000.0, 494000.0, 500000.0],
    use_geotop=False,
    cachename="combined_layer_ds.nc",
    cachedir=cachedir,
)
layer_model

## Clearing the cache

Sometimes you want to get rid of all the cached files to free disk space or to support your minimalistic lifestyle. You can use the `clear_cache` function to clear all cached files in a specific cache directory. Note: when calling this function an extra confirmation [Y/N] is required to actually delete all the cache files.

In [ ]:
# nlmod.cache.clear_cache(cachedir)

## Apply caching to a function
We have a simple function that takes some input and returns an xarray Dataset. We've put this function in the cache_example.py so we can import it in this notebook. In the module we used the `nlmod.cache.cache_netcdf` decorator on the function.

In [ ]:
from cache_example import func_to_create_a_dataset

Now we can call the function to create a simple Dataset.

In [ ]:
func_to_create_a_dataset(10)

We can also call this function using the `cachedir` and `cachename` arguments, even though we haven't defined these arguments in our function definition. The arguments were added by the decorator. 

In [ ]:
ds = func_to_create_a_dataset(10, cachedir=cachedir, cachename="example")
ds.close()
ds

Now the caching logic described in this notebook is applied to the function call. Next time we call this function it returns the cached dataset. Note that we use `ds.close()` to close the connection between the dataset and the netcdf file. If we do not close the connection you can get permission errors when you try to overwrite the file.

In [ ]:
ds = func_to_create_a_dataset(10, cachedir=cachedir, cachename="example")
ds.close()
ds

Or it creates a new cached dataset if our function arguments have been changed:

In [ ]:
func_to_create_a_dataset(11, cachedir=cachedir, cachename="example")
ds.close()
ds

When we decorate a function the cachedir and cachename arguments are also added to the docstring as you can see below:

In [ ]:
# show that the arguments cachedir and cachename are added to the docstring
?func_to_create_a_dataset

## Technicalities and discussion

In nlmod we use a specific caching method called [memoization](https://en.wikipedia.org/wiki/Memoization). The memoization is implemented in the `nlmod` caching module. The `cache_netcdf` decorator function handles most of the magic for caching netcdf files. When the cache is created all function arguments are stored in a dictionary and saved (pickled) as a .pklz file. The check on function arguments (step 2A) is done by reading the pickle and comparing the output with the arguments of the current function call.


### Notes

1. All function arguments are pickled and saved together with the netcdf file. If the function arguments use a lot of memory this process can be become slow. This should be taken into account when you decide to use caching.
2. Function arguments that cannot be pickled using the `pickle` module raise an error in the caching process.
3. If one of the function arguments is an xarray Dataset the argument check is somewhat different. Not all the data in a dataset it checked as it would take to much time. Therefore we use the `nlmod.cache.ds_contains` function to specify which coordinates, data variables and attributes should be checked. The arguments for this are specified in the cache decorator.
4. There is a check to see if the module where the function is defined has been changed since the cache was created. This helps not to use the cache when changes are made to the function. Unfortunately when the function uses other functions from different modules these other modules are not checked for recent changes.
5. The `cache_netcdf` decorator uses `functools.wraps` and some homemade magic to add properties, such as the name and the docstring, of the original function to the decorated function. This assumes that the original function has a docstring with a "Returns" heading. If this is not the case the docstring is not modified.
6. There are some additional options for caching in the NLMOD_CACHE_OPTIONS. The options are:
    - `nc_hash`: Save a hash of the stored netcdf file with the function arguments to ensure that the stored netcdf file and the pickled function arguments belong together. Default is True
    - `dataset_coords_hash`: Save a hash of the dataset coordinates with the function arguments and check against the dataset coordinates that are used to call the function. This is to ensure that the cached file was created using the same dataset coordinates as the current function call, default is True.
    - `dataset_data_vars_hash`: Save a hash of the data variables with the function arguments and check against the dataset variables that are used to call the function. This is to ensure that the cached file was created using the same dataset variables as the current function call, default is True.
    - `explicit_dataset_coordinate_comparison`: If the `dataset_coords_hash` and `dataset_data_vars_hash` do not work (for example see this issue: https://github.com/gwmod/nlmod/issues/389) it is adviced to use an explicit check to see if the cached file was created using the same dataset coordinates as the current function call. This can be done with this settings, the default is False.

In [ ]:
# show cache options
nlmod.config.NLMOD_CACHE_OPTIONS

In [ ]:
# set cache option
nlmod.config.set_options(nc_hash=False)

### Storing cache on disk

Many memoization methods use a hash of the function arguments as the filename. Thus creating multiple files for different function calls. The memoization in `nlmod` uses a user-defined filename (`cachename`) to store the cache. If the function is called with different arguments the previous cached file is overwritten. By not creating a new file for every unique set of function arguments we reduce the number of files and therefore the memory size on the disk. By saving the function output as netCDF file it is also possible to read the file seperately from the caching process. While this is not something you would often do it can help when debugging. 